In [1]:
import numpy as np
import pandas as pd
import glob
import sys

# loading some functions we wrote before
sys.path.append("/Users/jonathanramos/Desktop/LRI/Image ROI Data Wrangling/")
from clean import *
from norm import *
from count import *

In [2]:
df = pd.read_csv('RAM-12,13,14_FINAL.csv')
df_redo = pd.read_csv('RAM-REDO_FINAL.csv')

set(df.columns) - set(df_redo.columns)

{'CoM_x',
 'CoM_y',
 'dox',
 'grouping',
 'iid_rid',
 'mean_intensity',
 'rat_n',
 'react',
 'treatment'}

# Adding labels to redo set

In [3]:
# load in key, standardize cols and labels
df_key = pd.read_csv('RAM Ensembles Complete Cohort Key.csv')
df_key['Group'] = df_key.Group.replace('RAM-14_a', 'RAM-14a').replace('RAM-14_b', 'RAM-14b')
df_key['rat_n'] = df_key.apply(lambda x: '_'.join([x['Group'], str(x['Rat ID'])]), axis=1)
df_key['treatment'] = df_key['ABC/Veh'].str.upper()
df_key['dox'] = df_key['Dox Tx'].str.replace(' ', '_').str.upper()
df_key['react'] = df_key['React'].str.upper()

# select out the cols we are interested in
df_key = df_key[['rat_n', 'react', 'treatment', 'dox']]

### add missing rat information; this was confirmed by AG
df_key.loc[-1] = ['RAM-13_7', 'NR', 'ABC', 'OFF_DOX']
df_key.index = df_key.index + 1
df_key = df_key.sort_index()


def get_cols(df, df_key):
    # parse filename to build rat_n, image_name
    df['rat_n'] = df.filename.str.split('_').apply(lambda x: '_'.join(x[:2]))

    # merge with key on rat_n
    df = df.merge(df_key, how='left', on='rat_n')

    return df

df_redo = get_cols(df_redo, df_key)
df_redo = df_redo.rename(columns={'xm': 'CoM_x', 'ym': 'CoM_y', 'mean': 'mean_intensity', 'roi_id':'iid_rid', 'roi_id.1': 'roi_id'})

# Concat full set and redo set

In [4]:
df = df[['iid_rid', 'dummy_cFos', 'dummy_EGFP', 'dummy_mKate',
    'image_name', 'roi_id', 'stain', 'CoM_x', 'CoM_y', 'background',
    'mean_intensity', 'filename', 'rat_n', 'react', 'treatment', 'dox',
    'true_grouping']]

df_redo = df_redo[['iid_rid', 'dummy_cFos', 'dummy_EGFP', 'dummy_mKate',
    'image_name', 'roi_id', 'stain', 'CoM_x', 'CoM_y', 'background',
    'mean_intensity', 'filename', 'rat_n', 'react', 'treatment', 'dox',
    'true_grouping']]


df_full = pd.concat([df, df_redo])
print(f'shape: {df_full.shape}')

duplicates = df_full.duplicated(subset=['CoM_x', 'CoM_y', 'roi_id', 'image_name', 'mean_intensity', 'background']).sum()
print(f'\nduplicates: {duplicates}')

print('\nNaNs per col:')
print(df_full.isna().sum())
df_full.head()

shape: (43483, 17)

duplicates: 0

NaNs per col:
iid_rid            0
dummy_cFos         0
dummy_EGFP         0
dummy_mKate        0
image_name         0
roi_id             0
stain              0
CoM_x             31
CoM_y             31
background         0
mean_intensity    31
filename           0
rat_n              0
react              0
treatment          0
dox                0
true_grouping      0
dtype: int64


,iid_rid,dummy_cFos,dummy_EGFP,dummy_mKate,image_name,roi_id,stain,CoM_x,CoM_y,background,mean_intensity,filename,rat_n,react,treatment,dox,true_grouping
0,RAM-12_11_PFC_3.5_A_0-002-00000_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00000_mKate,mKate,490.69,263.66,54.2523,130.8234,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00028_EGFP', '0-002-00000_mKate')"
1,RAM-12_11_PFC_3.5_A_0-002-00001_mKate,False,False,True,RAM-12_11_PFC_3.5_A,0-002-00001_mKate,mKate,359.15,417.22,54.2523,252.4054,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-002-00001_mKate',)"
2,RAM-12_11_PFC_3.5_A_0-002-00002_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00002_mKate,mKate,149.95,354.17,54.2523,910.3442,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00047_EGFP', '0-002-00002_mKate')"
3,RAM-12_11_PFC_3.5_A_0-002-00003_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00003_mKate,mKate,193.05,160.84,54.2523,280.5944,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00128_EGFP', '0-002-00003_mKate')"
4,RAM-12_11_PFC_3.5_A_0-002-00004_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00004_mKate,mKate,221.09,254.73,54.2523,237.5106,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-FFF-00238_EGFP', '0-002-00004_mKate')"


In [5]:
# dropping those 31 rows containing nans
df_full = df_full.dropna()
print(f'shape: {df_full.shape}')

shape: (43452, 17)


## compute normalized mean-background

In [6]:
df_full['mean-background'] = df_full.mean_intensity - df_full.background
norm_condition = df_full.query('dox == "ON_DOX" and react == "FR1"')['mean-background'].mean()
df_full['norm_mean-background'] = df_full['mean-background'] / norm_condition

df_full

,iid_rid,dummy_cFos,dummy_EGFP,dummy_mKate,image_name,roi_id,stain,CoM_x,CoM_y,background,mean_intensity,filename,rat_n,react,treatment,dox,true_grouping,mean-background,norm_mean-background
0,RAM-12_11_PFC_3.5_A_0-002-00000_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00000_mKate,mKate,490.690,263.660,54.2523,130.8234,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00028_EGFP', '0-002-00000_mKate')",76.5711,0.518172
1,RAM-12_11_PFC_3.5_A_0-002-00001_mKate,False,False,True,RAM-12_11_PFC_3.5_A,0-002-00001_mKate,mKate,359.150,417.220,54.2523,252.4054,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-002-00001_mKate',)",198.1531,1.340941
2,RAM-12_11_PFC_3.5_A_0-002-00002_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00002_mKate,mKate,149.950,354.170,54.2523,910.3442,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00047_EGFP', '0-002-00002_mKate')",856.0919,5.793343
3,RAM-12_11_PFC_3.5_A_0-002-00003_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00003_mKate,mKate,193.050,160.840,54.2523,280.5944,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-01c-00128_EGFP', '0-002-00003_mKate')",226.3421,1.531702
4,RAM-12_11_PFC_3.5_A_0-002-00004_mKate,False,True,True,RAM-12_11_PFC_3.5_A,0-002-00004_mKate,mKate,221.090,254.730,54.2523,237.5106,RAM-12_11_PFC_3.5_A_4.tif,RAM-12_11,VR5,ABC,OFF_DOX,"('0-FFF-00238_EGFP', '0-002-00004_mKate')",183.2583,1.240145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,RAM-12_14_PFC_4.0_B_mKate_45,False,False,True,RAM-12_14_PFC_4.0_B,RAM-12_14_PFC_4.0_B_mKate_45,mKate,126.038,396.117,19.5310,221.3070,RAM-12_14_PFC_4.0_B_4,RAM-12_14,NR,VEH,OFF_DOX,"('RAM-12_14_PFC_4.0_B_mKate_45',)",201.7760,1.365458
2226,RAM-12_14_PFC_4.0_B_mKate_46,False,True,True,RAM-12_14_PFC_4.0_B,RAM-12_14_PFC_4.0_B_mKate_46,mKate,20.795,385.174,19.5310,513.8450,RAM-12_14_PFC_4.0_B_4,RAM-12_14,NR,VEH,OFF_DOX,"('RAM-12_14_PFC_4.0_B_EGFP_300', 'RAM-12_14_PF...",494.3140,3.345120
2227,RAM-12_14_PFC_4.0_B_mKate_47,False,True,True,RAM-12_14_PFC_4.0_B,RAM-12_14_PFC_4.0_B_mKate_47,mKate,48.338,409.231,19.5310,82.0070,RAM-12_14_PFC_4.0_B_4,RAM-12_14,NR,VEH,OFF_DOX,"('RAM-12_14_PFC_4.0_B_EGFP_299', 'RAM-12_14_PF...",62.4760,0.422787
2228,RAM-12_14_PFC_4.0_B_mKate_48,False,False,True,RAM-12_14_PFC_4.0_B,RAM-12_14_PFC_4.0_B_mKate_48,mKate,79.833,159.605,19.5310,71.3380,RAM-12_14_PFC_4.0_B_4,RAM-12_14,NR,VEH,OFF_DOX,"('RAM-12_14_PFC_4.0_B_mKate_48',)",51.8070,0.350588


# Counting Mean Cell ns

In [7]:
df_img_ns = df_full.groupby('rat_n')['image_name'].unique().reset_index()
df_img_ns['n_imgs'] = df_img_ns.image_name.apply(len)
df_img_ns

,rat_n,image_name,n_imgs
0,RAM-12_11,"[RAM-12_11_PFC_3.5_A, RAM-12_11_PFC_3.7_B, RAM...",4
1,RAM-12_12,"[RAM-12_12_PFC_3.4_A, RAM-12_12_PFC_3.4_B, RAM...",4
2,RAM-12_13,"[RAM-12_13_PFC_3.7_B, RAM-12_13_PFC_3.7_C, RAM...",4
3,RAM-12_14,"[RAM-12_14_PFC_3.8_C, RAM-12_14_PFC_3.8_D, RAM...",4
4,RAM-12_16,"[RAM-12_16_PFC_3.7_D, RAM-12_16_PFC_4.0_A, RAM...",4
5,RAM-12_3,"[RAM-12_3_PFC_3.7_A, RAM-12_3_PFC_3.7_B, RAM-1...",4
6,RAM-12_6,"[RAM-12_6_PFC_3.5_A, RAM-12_6_PFC_3.5_B, RAM-1...",3
7,RAM-12_7,"[RAM-12_7_PFC_3.5_C, RAM-12_7_PFC_3.5_D, RAM-1...",4
8,RAM-13_1,"[RAM-13_1_PFC_3.5_C, RAM-13_1_PFC_3.5_D, RAM-1...",4
9,RAM-13_3,"[RAM-13_3_PFC_3.3_D, RAM-13_3_PFC_3.5_E, RAM-1...",5


## Single cFos, single mKate

In [8]:
def mean_cell_ns_single(stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df_full to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_full.query(f'dummy_{stain_x} == True and stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_full[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox, then treatment, then react; reorder cols
    q = q.sort_values(by=['dox', 'treatment', 'react'])\
        [['rat_n', 'dox', 'treatment', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

# these should be identical, but i'm performing both queries for completeness
mean_cell_ns_single('cFos').to_csv('RAM-12,13,14_cFos_mean_cell_ns.csv')
mean_cell_ns_single('mKate').to_csv('RAM-12,13,14_mKate_mean_cell_ns.csv')

## cFos on mKate, mKate on cFos

In [9]:
def mean_cell_ns_double(stain_x, stain_y):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df_full to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_full.query(f'dummy_{stain_x} == True and dummy_{stain_y} == True and stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_full[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox, then treatment, then react; reorder cols
    q = q.sort_values(by=['dox', 'treatment', 'react'])\
        [['rat_n', 'dox', 'treatment', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    return q

# these should be identical, but i'm performing both queries for completeness
mean_cell_ns_double('cFos', 'mKate').to_csv('RAM-12,13,14_cFos_coloc_w_mKate_mean_cell_ns.csv')
mean_cell_ns_double('mKate', 'cFos').to_csv('RAM-12,13,14_mKate_coloc_w_cFos_mean_cell_ns.csv')

In [10]:
mean_cell_ns_double('cFos', 'mKate')

,rat_n,dox,treatment,react,sum,n_imgs,mean_cell_n
0,RAM-12_12,OFF_DOX,ABC,FR1,22,4,5.500000
1,RAM-14b_6,OFF_DOX,ABC,FR1,2,2,1.000000
2,RAM-14b_7,OFF_DOX,ABC,FR1,1,1,1.000000
3,RAM-14b_8,OFF_DOX,ABC,FR1,57,4,14.250000
4,RAM-13_7,OFF_DOX,ABC,NR,62,4,15.500000
5,RAM-14b_2,OFF_DOX,ABC,NR,62,4,15.500000
6,RAM-14b_4,OFF_DOX,ABC,NR,85,4,21.250000
7,RAM-12_11,OFF_DOX,ABC,VR5,9,4,2.250000
8,RAM-12_13,OFF_DOX,VEH,FR1,80,4,20.000000
9,RAM-13_8,OFF_DOX,VEH,FR1,85,5,17.000000


# Repeat cell ns, but this time with split by cFos intensity (median split)

## cFos_hi

In [11]:
df_cFos = df_full.query('dummy_cFos == True and stain =="cFos"').copy()
cFos_median = df_cFos['norm_mean-background'].median()
df_cFos['cFos_split'] = df_cFos['norm_mean-background'].apply(lambda x: 'cFos_hi' if x > cFos_median else 'cFos_lo')

q = df_cFos.query('cFos_split == "cFos_hi"')\
    .groupby(['cFos_split', 'image_name'])['rat_n'].count()\
    .reset_index().rename(columns={'rat_n':'count'})\
    .merge(df_cFos[['image_name','rat_n']], on='image_name', how='left')\
    .drop_duplicates()\
    .groupby(['rat_n', 'cFos_split'])['count'].sum().reset_index().rename(columns={'count':'sum'})\
    .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
        on='rat_n', how='left')\
    .drop_duplicates().reset_index()\
    .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

# compute means
q['mean_cell_n'] = q['sum'] / q['n_imgs']

# sort by dox, then treatment, then react; reorder cols
q = q.sort_values(by=['dox', 'treatment', 'react'])\
    [['rat_n', 'dox', 'treatment', 'react', 'cFos_split', 'sum', 'n_imgs', 'mean_cell_n']]\
    .reset_index().drop('index', axis=1)

q.to_csv('RAM-12,13,14_cFos_hi_mean_cell_ns.csv')
q

,rat_n,dox,treatment,react,cFos_split,sum,n_imgs,mean_cell_n
0,RAM-12_12,OFF_DOX,ABC,FR1,cFos_hi,253,4,63.250000
1,RAM-14b_6,OFF_DOX,ABC,FR1,cFos_hi,74,2,37.000000
2,RAM-14b_7,OFF_DOX,ABC,FR1,cFos_hi,62,1,62.000000
3,RAM-14b_8,OFF_DOX,ABC,FR1,cFos_hi,351,4,87.750000
4,RAM-13_7,OFF_DOX,ABC,NR,cFos_hi,464,4,116.000000
5,RAM-14b_2,OFF_DOX,ABC,NR,cFos_hi,495,4,123.750000
6,RAM-14b_4,OFF_DOX,ABC,NR,cFos_hi,770,4,192.500000
7,RAM-12_11,OFF_DOX,ABC,VR5,cFos_hi,88,4,22.000000
8,RAM-12_13,OFF_DOX,VEH,FR1,cFos_hi,290,4,72.500000
9,RAM-13_8,OFF_DOX,VEH,FR1,cFos_hi,298,5,59.600000


## cFos_lo

In [12]:
q = df_cFos.query('cFos_split == "cFos_lo"')\
    .groupby(['cFos_split', 'image_name'])['rat_n'].count()\
    .reset_index().rename(columns={'rat_n':'count'})\
    .merge(df_cFos[['image_name','rat_n']], on='image_name', how='left')\
    .drop_duplicates()\
    .groupby(['rat_n', 'cFos_split'])['count'].sum().reset_index().rename(columns={'count':'sum'})\
    .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
        on='rat_n', how='left')\
    .drop_duplicates().reset_index()\
    .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

# compute means
q['mean_cell_n'] = q['sum'] / q['n_imgs']

# sort by dox, then treatment, then react; reorder cols
q = q.sort_values(by=['dox', 'treatment', 'react'])\
    [['rat_n', 'dox', 'treatment', 'react', 'cFos_split', 'sum', 'n_imgs', 'mean_cell_n']]\
    .reset_index().drop('index', axis=1)

q.to_csv('RAM-12,13,14_cFos_lo_mean_cell_ns.csv')
q

,rat_n,dox,treatment,react,cFos_split,sum,n_imgs,mean_cell_n
0,RAM-12_12,OFF_DOX,ABC,FR1,cFos_lo,187,4,46.750000
1,RAM-14b_6,OFF_DOX,ABC,FR1,cFos_lo,122,2,61.000000
2,RAM-14b_7,OFF_DOX,ABC,FR1,cFos_lo,64,1,64.000000
3,RAM-14b_8,OFF_DOX,ABC,FR1,cFos_lo,409,4,102.250000
4,RAM-13_7,OFF_DOX,ABC,NR,cFos_lo,816,4,204.000000
5,RAM-14b_2,OFF_DOX,ABC,NR,cFos_lo,456,4,114.000000
6,RAM-14b_4,OFF_DOX,ABC,NR,cFos_lo,295,4,73.750000
7,RAM-12_11,OFF_DOX,ABC,VR5,cFos_lo,27,4,6.750000
8,RAM-12_13,OFF_DOX,VEH,FR1,cFos_lo,425,4,106.250000
9,RAM-13_8,OFF_DOX,VEH,FR1,cFos_lo,518,5,103.600000


In [13]:
q = df_cFos.query('cFos_split == "cFos_hi" and dummy_mKate == True')\
    .groupby(['cFos_split', 'image_name'])['rat_n'].count()\
    .reset_index().rename(columns={'rat_n':'count'})\
    .merge(df_cFos[['image_name','rat_n']], on='image_name', how='left')\
    .drop_duplicates()\
    .groupby(['rat_n', 'cFos_split'])['count'].sum().reset_index().rename(columns={'count':'sum'})\
    .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
        on='rat_n', how='left')\
    .drop_duplicates().reset_index()\
    .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

# compute means
q['mean_cell_n'] = q['sum'] / q['n_imgs']

# sort by dox, then treatment, then react; reorder cols
q = q.sort_values(by=['dox', 'treatment', 'react'])\
    [['rat_n', 'dox', 'treatment', 'react', 'cFos_split', 'sum', 'n_imgs', 'mean_cell_n']]\
    .reset_index().drop('index', axis=1)

# fill in rats who did not have this stain type combination with 0
missing = set(df_img_ns.rat_n) - set(q.rat_n)
df_missing = df_key[df_key.rat_n.isin(missing)].copy()
df_missing['cFos_split'] = 'cFos_hi'
df_missing['sum'] = 0
df_missing = df_missing.merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')
df_missing['mean_cell_n'] = df_missing['sum'] / df_missing['n_imgs']
df_missing
q = pd.concat([q, df_missing]).sort_values(by=['dox', 'treatment', 'react']).reset_index().drop('index', axis=1)

q.to_csv('RAM-12,13,14_cFos_hi_coloc_w_mKate_mean_cell_ns.csv')
q

,rat_n,dox,treatment,react,cFos_split,sum,n_imgs,mean_cell_n
0,RAM-12_12,OFF_DOX,ABC,FR1,cFos_hi,16,4,4.000000
1,RAM-14b_6,OFF_DOX,ABC,FR1,cFos_hi,2,2,1.000000
2,RAM-14b_8,OFF_DOX,ABC,FR1,cFos_hi,31,4,7.750000
3,RAM-14b_7,OFF_DOX,ABC,FR1,cFos_hi,0,1,0.000000
4,RAM-13_7,OFF_DOX,ABC,NR,cFos_hi,39,4,9.750000
5,RAM-14b_2,OFF_DOX,ABC,NR,cFos_hi,43,4,10.750000
6,RAM-14b_4,OFF_DOX,ABC,NR,cFos_hi,72,4,18.000000
7,RAM-12_11,OFF_DOX,ABC,VR5,cFos_hi,6,4,1.500000
8,RAM-12_13,OFF_DOX,VEH,FR1,cFos_hi,38,4,9.500000
9,RAM-13_8,OFF_DOX,VEH,FR1,cFos_hi,36,5,7.200000


In [14]:
q = df_cFos.query('cFos_split == "cFos_lo" and dummy_mKate == True')\
    .groupby(['cFos_split', 'image_name'])['rat_n'].count()\
    .reset_index().rename(columns={'rat_n':'count'})\
    .merge(df_cFos[['image_name','rat_n']], on='image_name', how='left')\
    .drop_duplicates()\
    .groupby(['rat_n', 'cFos_split'])['count'].sum().reset_index().rename(columns={'count':'sum'})\
    .merge(df_full[['rat_n', 'react', 'treatment', 'dox']],\
        on='rat_n', how='left')\
    .drop_duplicates().reset_index()\
    .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

# compute means
q['mean_cell_n'] = q['sum'] / q['n_imgs']

# sort by dox, then treatment, then react; reorder cols
q = q.sort_values(by=['dox', 'treatment', 'react'])\
    [['rat_n', 'dox', 'treatment', 'react', 'cFos_split', 'sum', 'n_imgs', 'mean_cell_n']]\
    .reset_index().drop('index', axis=1)

# fill in rats who did not have this stain type combination with 0
missing = set(df_img_ns.rat_n) - set(q.rat_n)
df_missing = df_key[df_key.rat_n.isin(missing)].copy()
df_missing['cFos_split'] = 'cFos_hi'
df_missing['sum'] = 0
df_missing = df_missing.merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')
df_missing['mean_cell_n'] = df_missing['sum'] / df_missing['n_imgs']
df_missing
q = pd.concat([q, df_missing]).sort_values(by=['dox', 'treatment', 'react']).reset_index().drop('index', axis=1)

q.to_csv('RAM-12,13,14_cFos_lo_coloc_w_mKate_mean_cell_ns.csv')
q

,rat_n,dox,treatment,react,cFos_split,sum,n_imgs,mean_cell_n
0,RAM-12_12,OFF_DOX,ABC,FR1,cFos_lo,6,4,1.50
1,RAM-14b_7,OFF_DOX,ABC,FR1,cFos_lo,1,1,1.00
2,RAM-14b_8,OFF_DOX,ABC,FR1,cFos_lo,26,4,6.50
3,RAM-14b_6,OFF_DOX,ABC,FR1,cFos_hi,0,2,0.00
4,RAM-13_7,OFF_DOX,ABC,NR,cFos_lo,23,4,5.75
5,RAM-14b_2,OFF_DOX,ABC,NR,cFos_lo,19,4,4.75
6,RAM-14b_4,OFF_DOX,ABC,NR,cFos_lo,13,4,3.25
7,RAM-12_11,OFF_DOX,ABC,VR5,cFos_lo,3,4,0.75
8,RAM-12_13,OFF_DOX,VEH,FR1,cFos_lo,42,4,10.50
9,RAM-13_8,OFF_DOX,VEH,FR1,cFos_lo,49,5,9.80
